In [20]:
import os
import pandas as pd

# Dictionary to store dataframes with filenames as keys
dataframes = {}

# Path to the data folder
data_folder = 'data/'

# Loop through all files in the data folder
for filename in os.listdir(data_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)
        dataframes[filename[:-4]] = df


- Data from 2022
- Keep only users that have both biometrics and exercise data
- Drop records where we dont have a timestamp
- Drop duplicates

In [21]:
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
dataframes['ex1']

,CloudId,Gender,Age,PhysicalActivityMacroTypeName,ExerciseName,EquipmentName,DoneOnUTC,Duration_sec,Calories,MetsMin
0,ed958cce9812f1b8607f8c8ab5836c1f450f858b,M,13,Cardio,Custom exercise in time,Bike Artis,2022-01-23 10:00:31.425 UTC,215,30.0,5.6
1,ed958cce9812f1b8607f8c8ab5836c1f450f858b,M,13,Cardio,GOAL exercise in time,Run Artis,2022-01-23 10:09:01.153 UTC,215,29.0,5.3
2,ed958cce9812f1b8607f8c8ab5836c1f450f858b,M,13,Isotonic,Leg curl,Leg Curl Biostr,2022-01-23 10:28:49.724 UTC,85,9.0,4.0
3,56a55ec8f237732fff2239735b469e346c8a5f6b,M,17,Cardio,GOAL exercise in time,Run,2022-01-23 08:52:44.24 UTC,600,90.0,8.4
4,926bd1a7bbf9ccd40101befc6d409e2aa5979fd2,F,17,Cardio,GOAL exercise in time,Run,2022-01-23 17:13:38.84 UTC,660,126.0,8.6
...,...,...,...,...,...,...,...,...,...,...
1039190,14b8cde284d65e69d081f4a683a95dcdd70b6d10,F,51,Isotonic,Hip mobility,Wellness Ball Active Sitting,2022-03-30 05:30:30.444 UTC,60,4.0,5.0
1039191,14b8cde284d65e69d081f4a683a95dcdd70b6d10,F,51,Isotonic,Ab stretch - legs bent,Wellness Ball Active Sitting,2022-03-30 05:26:44.718 UTC,60,2.0,2.5
1039192,14b8cde284d65e69d081f4a683a95dcdd70b6d10,F,51,Isotonic,Pelvic tilt & low back mobility - feet lifted,Wellness Ball Active Sitting,2022-03-30 05:25:41.653 UTC,60,4.0,5.0
1039193,14b8cde284d65e69d081f4a683a95dcdd70b6d10,F,51,Isotonic,Anterior pelvic tilt - seated,Wellness Ball Active Sitting,2022-03-30 05:34:33.52 UTC,60,4.0,5.0


In [23]:
for name, df in dataframes.items():
    print(f"Columns in dataframe '{name}':")
    print(df.columns)
    print()

Columns in dataframe 'bio2':
Index(['CloudId', 'Gender', 'Age', 'BiometricName', 'MeasureProvidedBy',
       'MeasuredOnUTC', 'Value'],
      dtype='object')

Columns in dataframe 'bio1':
Index(['CloudId', 'Gender', 'Age', 'BiometricName', 'MeasureProvidedBy',
       'MeasuredOnUTC', 'Value'],
      dtype='object')

Columns in dataframe 'ex1':
Index(['CloudId', 'Gender', 'Age', 'PhysicalActivityMacroTypeName',
       'ExerciseName', 'EquipmentName', 'DoneOnUTC', 'Duration_sec',
       'Calories', 'MetsMin'],
      dtype='object')

Columns in dataframe 'ex2':
Index(['CloudId', 'Gender', 'Age', 'PhysicalActivityMacroTypeName',
       'ExerciseName', 'ExerciseBodyPartName', 'ExerciseMusclesName',
       'EquipmentName', 'DoneOnUTC', 'Duration_sec', 'Calories', 'METs'],
      dtype='object')



### Check ranges

In [24]:
# Concatenate the dataframes
biometrics = pd.concat([dataframes['bio1'], dataframes['bio2']])

# Convert the 'MeasuredOnUTC' column to datetime
biometrics['MeasuredOnUTC'] = pd.to_datetime(biometrics['MeasuredOnUTC'], errors='coerce')

# Drop rows with NaT in 'MeasuredOnUTC'
biometrics = biometrics.dropna(subset=['MeasuredOnUTC'])

# Get the time range
time_range = (biometrics['MeasuredOnUTC'].min(), biometrics['MeasuredOnUTC'].max())

print(f"Time range for the combined dataframe: {time_range[0]} to {time_range[1]}")

Time range for the combined dataframe: 2022-01-02 01:34:04.924000+00:00 to 2022-12-31 23:27:32.374000+00:00


In [7]:
# Concatenate the dataframes
exercises = pd.concat([dataframes['ex1'], dataframes['ex2']])

# Convert the 'MeasuredOnUTC' column to datetime
exercises['DoneOnUTC'] = pd.to_datetime(exercises['DoneOnUTC'], errors='coerce')

# Drop rows with NaT in 'MeasuredOnUTC'
exercises = exercises.dropna(subset=['DoneOnUTC'])

# Get the time range
time_range = (exercises['DoneOnUTC'].min(), exercises['DoneOnUTC'].max())

print(f"Time range for the combined dataframe: {time_range[0]} to {time_range[1]}")

Time range for the combined dataframe: 2022-01-02 00:00:45.421000+00:00 to 2022-12-31 23:59:42.223000+00:00


### Keep only users for which we have both biometrics and exercise data

In [25]:
# Count the number of distinct users in the exercises dataframe
distinct_users_exercises = exercises['CloudId'].nunique()
print(f"Number of distinct users in exercises dataframe: {distinct_users_exercises}")

# Count the number of distinct users in the biometrics dataframe
distinct_users_biometrics = biometrics['CloudId'].nunique()
print(f"Number of distinct users in biometrics dataframe: {distinct_users_biometrics}")

Number of distinct users in exercises dataframe: 6360
Number of distinct users in biometrics dataframe: 8312


In [26]:
# Find common users in both dataframes
common_users = set(biometrics['CloudId']).intersection(set(exercises['CloudId']))

# Filter both dataframes to keep only records with common users
biometrics_common = biometrics[biometrics['CloudId'].isin(common_users)]
exercises_common = exercises[exercises['CloudId'].isin(common_users)]

# Count the number of common users
num_common_users = len(common_users)
print(f"Number of common users in both dataframes: {num_common_users}")

Number of common users in both dataframes: 6360


In [27]:
# Filter both dataframes to keep only records with common users
biometrics_filtered = biometrics[biometrics['CloudId'].isin(common_users)]
exercises_filtered = exercises[exercises['CloudId'].isin(common_users)]

print(f"Filtered biometrics dataframe shape: {biometrics_filtered.shape}")
print(f"Filtered exercises dataframe shape: {exercises_filtered.shape}")

Filtered biometrics dataframe shape: (4900329, 7)
Filtered exercises dataframe shape: (3315940, 13)


#### Drop duplicates

In [28]:
# Remove duplicated rows in biometrics_filtered
biometrics_filtered = biometrics_filtered.drop_duplicates()

# Remove duplicated rows in exercises_filtered
exercises_filtered = exercises_filtered.drop_duplicates()

print(f"biometrics_filtered shape after removing duplicates: {biometrics_filtered.shape}")
print(f"exercises_filtered shape after removing duplicates: {exercises_filtered.shape}")

biometrics_filtered shape after removing duplicates: (4889207, 7)
exercises_filtered shape after removing duplicates: (3315940, 13)


In [29]:
# Update exercises and biometrics with the filtered dataframes
exercises = exercises_filtered
biometrics = biometrics_filtered

print(f"Updated exercises dataframe shape: {exercises.shape}")
print(f"Updated biometrics dataframe shape: {biometrics.shape}")

Updated exercises dataframe shape: (3315940, 13)
Updated biometrics dataframe shape: (4889207, 7)


In [30]:
print(exercises['CloudId'].nunique())
print(biometrics['CloudId'].nunique())

6360
6360


### Handle null values

In [31]:
# Find the number of null values in each dataframe
null_values_biometrics = biometrics.isnull().sum()
null_values_exercises = exercises.isnull().sum()

print("Null values in biometrics dataframe:")
print(null_values_biometrics)
print("\nNull values in exercises dataframe:")
print(null_values_exercises)

Null values in biometrics dataframe:
CloudId                    0
Gender                     0
Age                        0
BiometricName              0
MeasureProvidedBy    4885732
MeasuredOnUTC              0
Value                      0
dtype: int64

Null values in exercises dataframe:
CloudId                                0
Gender                                 0
Age                                    0
PhysicalActivityMacroTypeName          0
ExerciseName                           0
EquipmentName                          3
DoneOnUTC                              0
Duration_sec                           0
Calories                               3
MetsMin                          2456592
ExerciseBodyPartName              862737
ExerciseMusclesName               863014
METs                              870238
dtype: int64


In [32]:
# Save the biometrics dataframe to a CSV file
biometrics.to_csv('biometrics_cleaned.csv', index=False)

# Save the exercises dataframe to a CSV file
exercises.to_csv('exercises_cleaned.csv', index=False)